# <center><span style='color:green'> Household Electricity Consumption </span></center>

## Problem Statement

* Predict the household electricity consumption
* Regression problem

## 1 <span style='color:red'>|</span> Import Libraries

In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#for modeling
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score , r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV

#database
import pymongo

#save the model
import pickle

#to read data
import zipfile

## 2 <span style='color:red'>|</span> Load Dataset

In [25]:
zip_file_path = "cosumption_data.zip"

with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    df = pd.read_csv("household_power_consumption.txt", delimiter=';')

In [26]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [27]:
df.shape

(2075259, 9)

* Taking a sample of 60000 as data is too large 

In [4]:
df_sample = df.sample(60000)
df_sample.shape

(60000, 9)

## 3 <span style='color:red'>|</span> Exploratory Data Analysis

In [5]:
df_sample.columns

Index(['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3'],
      dtype='object')

In [6]:
df_sample.drop(['Date','Time'], axis=1, inplace=True)

In [7]:
df_sample.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1233438,2.632,0.104,240.230,10.800,1.000,0.000,18.0
1309607,?,?,?,?,?,?,NaN
817152,0.728,0.188,241.320,3.000,0.000,0.000,1.0
1623532,0.250,0.000,247.450,1.000,0.000,0.000,0.0
1193085,0.446,0.178,243.280,2.000,0.000,1.000,0.0


In [8]:
df_sample.sample(10)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1831087,1.348,0.000,240.750,5.600,0.000,0.000,19.0
1003692,1.396,0.152,235.610,6.000,0.000,0.000,0.0
81923,1.526,0.096,240.390,6.200,1.000,1.000,17.0
491388,0.544,0.264,247.820,2.400,1.000,0.000,0.0
1968,0.304,0.142,246.490,1.400,0.000,0.000,0.0
376741,3.448,0.068,235.970,14.600,37.000,0.000,18.0
1799605,1.446,0.084,241.760,6.000,0.000,0.000,19.0
120650,1.868,0.590,239.640,8.000,0.000,2.000,18.0
1081550,1.676,0.106,244.660,6.800,0.000,0.000,0.0
465255,4.692,0.160,234.310,20.000,0.000,37.000,16.0


In [9]:
df_sample.dtypes

Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object

#### 3.1 <span style='color:red'>|</span> Check any special character and handle them 

In [10]:
character_check = any(not value.isnumeric() for value in df_sample['Global_active_power'])
character_check

True

In [11]:
character = [value for value in df_sample['Global_active_power'] if not str(value).replace('.','',1).isnumeric()]
set(character)

{'?'}

In [12]:
#Check character in dataframe
special_char = df_sample[df_sample['Global_active_power']=='?']
special_char

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1309607,?,?,?,?,?,?,NaN
1619197,?,?,?,?,?,?,NaN
1986212,?,?,?,?,?,?,NaN
1310951,?,?,?,?,?,?,NaN
1397743,?,?,?,?,?,?,NaN
...,...,...,...,...,...,...,...
1984992,?,?,?,?,?,?,NaN
1930857,?,?,?,?,?,?,NaN
1934797,?,?,?,?,?,?,NaN
1619919,?,?,?,?,?,?,NaN


* 737/60000 rows and 7/7 columns has special character in it

In [13]:
(737/60000)*100

1.2283333333333333

In [14]:
#drop the data having special character
print("Data before removal of special character :", df_sample.shape)
df_sample.drop(special_char.index, axis=0, inplace=True)
print("Data after removal of special character :", df_sample.shape)

Data before removal of special character : (60000, 7)
Data after removal of special character : (59257, 7)


#### 3.2 <span style='color:red'>|</span> Check duplicates

In [15]:
df_sample.duplicated().sum()

263

* 265 duplicates record found

In [16]:
print("Data with duplicate records :", df_sample.shape)
df_sample.drop_duplicates(inplace=True)
print("Data withour duplicate records :", df_sample.shape)

Data with duplicate records : (59257, 7)
Data withour duplicate records : (58994, 7)


#### 3.3 <span style='color:red'>|</span> Check null values

In [17]:
df_sample.isnull().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

* No null values in dataset

#### 3.4 <span style='color:red'>|</span> Convert dtype of columns

In [18]:
df_sample = df_sample.astype(float)

In [19]:
df_sample.dtypes

Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtype: object

#### 3.5 <span style='color:red'>|</span> Combining Features

In [20]:
df_sample['meter'] = df_sample['Sub_metering_1'] + df_sample['Sub_metering_2'] + df_sample['Sub_metering_2']

In [21]:
df_sample.drop(['Sub_metering_1','Sub_metering_2','Sub_metering_3'], axis=1, inplace=True)

In [22]:
df_sample.columns

Index(['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'meter'],
      dtype='object')

In [28]:
df_sample.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,meter
1233438,2.632,0.104,240.23,10.8,1.0
817152,0.728,0.188,241.32,3.0,0.0
1623532,0.250,0.000,247.45,1.0,0.0
1193085,0.446,0.178,243.28,2.0,2.0
1321839,0.294,0.226,240.15,1.4,2.0


In [29]:
df_sample.describe().T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,58994.0,1.098659,1.066037,0.076,0.310,0.608,1.534,9.338
Global_reactive_power,58994.0,0.123723,0.112490,0.000,0.048,0.100,0.194,1.190
Voltage,58994.0,240.820611,3.260856,223.490,238.960,240.970,242.890,253.230
Global_intensity,58994.0,4.656969,4.481814,0.200,1.400,2.600,6.400,41.800
meter,58994.0,3.784080,13.551102,0.000,0.000,0.000,2.000,179.000


#### 3.6 <span style='color:red'>|</span> Graphical Representation

In [ ]:
#Data Distribution in each column
plt.figure(figsize=(20,20), facecolor='white')